Practice Activity 

In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

13.2.5

Consider four possible models for predicting house prices:

Using only the size and number of rooms.
Using size, number of rooms, and building type.
Using size and building type, and their interaction.
Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.
Set up a pipeline for each of these four models.

Then, get predictions on the test set for each of your pipelines, and compute the root mean squared error. Which model performed best?

Note: You should only use the function train_test_split() one time in your code; that is, we should be predicting on the same test set for all three models.

In [3]:
ames = pd.read_csv("C:/Users/jonat/Documents/Calpoly MSBA/Fall/Machine Learning 544/Data/AmesHousing.csv")
ames.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,Fa,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,NaN,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,Ex,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [4]:
pd.Series(ames.columns)

0              Order
1                PID
2        MS SubClass
3          MS Zoning
4       Lot Frontage
           ...      
77           Mo Sold
78           Yr Sold
79         Sale Type
80    Sale Condition
81         SalePrice
Length: 82, dtype: object

In [5]:
X = ames.drop(columns=["SalePrice"])
y = ames["SalePrice"]

X_train, X_test,y_train,y_test = train_test_split(X,y, random_state=321)

In [6]:
lr_pipeline1 = Pipeline(
  [("standardize", StandardScaler()),
  ("linear_regression", LinearRegression())]
)

lr_pipeline1

Pipeline(steps=[('standardize', StandardScaler()),
                ('linear_regression', LinearRegression())])

In [7]:
# Model 1 (Size + Number Rooms)
X1 = ["Gr Liv Area","TotRms AbvGrd"]
Model1 = lr_pipeline1.fit(X_train[X1], y_train)

y_pred1 = Model1.predict(X_test[X1])
rmse1 = np.sqrt(mean_squared_error(y_test,y_pred1))

In [8]:
ct2 = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop"
)

lr_pipeline2 = Pipeline(
  [("preprocessing", ct2),
  ("linear_regression", LinearRegression())]
)

lr_pipeline2

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('dummify',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Bldg Type']),
                                                 ('standardize',
                                                  StandardScaler(),
                                                  ['Gr Liv Area',
                                                   'TotRms AbvGrd'])])),
                ('linear_regression', LinearRegression())])

In [9]:
# Model 2 (Size + Number of Rooms + Building Type)
X2 = ["Gr Liv Area", "TotRms AbvGrd","Bldg Type"]

Model2 = lr_pipeline2.fit(X_train[X2], y_train)

y_pred2 = Model2.predict(X_test[X2])
rmse2 = np.sqrt(mean_squared_error(y_test,y_pred2))

In [10]:
ct_inter = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("standardize", StandardScaler(), ["Gr Liv Area"]),
  ],
  remainder = "drop"
)

lr_pipeline3 = Pipeline(
  [("preprocessing", ct_inter),
  ("interaction", PolynomialFeatures(interaction_only = True)),
  ("linear_regression", LinearRegression())]
)

lr_pipeline3

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('dummify',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Bldg Type']),
                                                 ('standardize',
                                                  StandardScaler(),
                                                  ['Gr Liv Area'])])),
                ('interaction', PolynomialFeatures(interaction_only=True)),
                ('linear_regression', LinearRegression())])

In [11]:
# Model 3 (Size + Building Type + Interaction)

X3 = ["Gr Liv Area","Bldg Type"]

Model3 = lr_pipeline3.fit(X_train[X3], y_train)

y_pred3 = Model3.predict(X_test[X3])
rmse3 = np.sqrt(mean_squared_error(y_test,y_pred3))

In [12]:
ct_poly = ColumnTransformer(
    [
        ("dummify", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False), ["Bldg Type"]),
        ("polynomial", PolynomialFeatures(degree=5), ["Gr Liv Area", "TotRms AbvGrd"]),
    ],
    remainder="drop",
)


lr_pipeline4 = Pipeline(
    [
        ("preprocessing", ct_poly),
        ("linear_regression", LinearRegression()),
    ]
)



In [13]:
# Model 4 (5 Degree polynomial Size + # Rooms + Building Type)

X4 = ["Gr Liv Area","Bldg Type", "TotRms AbvGrd"]

Model4 = lr_pipeline4.fit(X_train[X4], y_train)

y_pred4 = Model4.predict(X_test[X4])
rmse4 = np.sqrt(mean_squared_error(y_test,y_pred4))

In [14]:
# Test
Models = ("Model 1", "Model 2","Model 3","Model 4")
RMSE= np.round([rmse1,rmse2,rmse3,rmse4],3)

rmse_df = pd.DataFrame({"Models":Models,"RMSE":RMSE})
rmse_df.sort_values("RMSE",ascending=True)

,Models,RMSE
2,Model 3,50828.667
1,Model 2,51375.113
0,Model 1,52703.664
3,Model 4,53041.508


Model 3 which included (size + building type + the interaction) performed the best with the lowest RMSE at 50,828.68

13.3.2

Once again consider four modeling options for house price:

Using only the size and number of rooms.
Using size, number of rooms, and building type.
Using size and building type, and their interaction.
Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.
Use cross_val_score with the pipelines you made earlier to find the cross-validated root mean squared error for each model.

Which do you prefer? Does this agree with your conclusion from earlier?

In [15]:
scores1 = cross_val_score(lr_pipeline1, X[X1], y, cv=5, scoring='neg_root_mean_squared_error')
scores2 = cross_val_score(lr_pipeline2, X[X2], y, cv=5, scoring='neg_root_mean_squared_error')
scores3 = cross_val_score(lr_pipeline3, X[X3], y, cv=5, scoring='neg_root_mean_squared_error')
scores4 = cross_val_score(lr_pipeline4, X[X4], y, cv=5, scoring='neg_root_mean_squared_error')

rmse_cross1 = -np.mean(scores1)
rmse_cross2 = -np.mean(scores2)
rmse_cross3 = -np.mean(scores3)
rmse_cross4 = -np.mean(scores4)



In [16]:
Models = ("Model 1", "Model 2","Model 3","Model 4")
RMSE= np.round([rmse_cross1,rmse_cross2,rmse_cross3,rmse_cross4],3)

rmse_cross_df = pd.DataFrame({"Models":Models,"RMSE":RMSE})
rmse_cross_df.sort_values("RMSE",ascending=True)

,Models,RMSE
2,Model 3,53430.922
1,Model 2,54168.081
0,Model 1,55806.326
3,Model 4,65602.790


Model 3 is still the best model even with cross validation at 53,430.92.

13.3.3

Consider one hundred modeling options for house price:

House size, trying degrees 1 through 10
Number of rooms, trying degrees 1 through 10
Building Type
Hint: The dictionary of possible values that you make to give to GridSearchCV will have two elements instead of one.

Q1: Which model performed the best?

Q2: What downsides do you see of trying all possible model options? How might you go about choosing a smaller number of tuning values to try?

In [17]:
ct_poly2 = ColumnTransformer(
    [
        ("poly_size",  PolynomialFeatures(), ["Gr Liv Area"]),
        ("poly_rooms", PolynomialFeatures(), ["TotRms AbvGrd"]),
        ("dummify",    OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False), ["Bldg Type"]),
    ],
    remainder="drop"
)

poly_pipeline = Pipeline([
    ("preprocessing", ct_poly2),
    ("linear_regression", LinearRegression())
])


In [18]:
param_grid = {
    "preprocessing__poly_size__degree": np.arange(1, 11),
    "preprocessing__poly_rooms__degree": np.arange(1, 11)
}


In [19]:
grid = GridSearchCV(poly_pipeline,param_grid,cv=5,scoring="neg_root_mean_squared_error")

grid.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('poly_size',
                                                                         PolynomialFeatures(),
                                                                         ['Gr '
                                                                          'Liv '
                                                                          'Area']),
                                                                        ('poly_rooms',
                                                                         PolynomialFeatures(),
                                                                         ['TotRms '
                                                                          'AbvGrd']),
                                                                        ('dummify',
                                                                         OneHotEncoder(drop='first',
                                                                                       handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         ['Bldg '
                                                                          'Type'])])),
                                       ('linear_regression',
                                        LinearRegression())]),
             param_grid={'preprocessing__poly_rooms__degree': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'preprocessing__poly_size__degree': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])},
             scoring='neg_root_mean_squared_error')

In [22]:
results = pd.DataFrame({
    "size_degree":  grid.cv_results_["param_preprocessing__poly_size__degree"].astype(int),
    "rooms_degree": grid.cv_results_["param_preprocessing__poly_rooms__degree"].astype(int),
    "mean_score":   grid.cv_results_["mean_test_score"].round(3)
})

len(results)

100

In [23]:
results["rmse"] = -results["mean_score"].round(3)
results = results.drop(columns="mean_score")
results.sort_values("rmse").head(10)

,size_degree,rooms_degree,rmse
2,3,1,52781.984
12,3,2,52837.446
22,3,3,53003.265
32,3,4,53163.134
62,3,7,53168.853
42,3,5,53386.552
52,3,6,53499.108
30,1,4,53596.302
40,1,5,53614.998
1,2,1,53925.418


Q1: The model that performed the best was a polynomial degree 3 for size and first degree polynomial for rooms leading to a rmse of 52,781.98 which is slightly better than Model 3 with cross validation at 53,430.92

Q2: Trying all possible combinations takes a large amount of time and computing power, if we included more predictors then the amount of degrees we have to check increases by a a factor of 10 each time. It would be better to see if certain predictor variables are significant or lower rmse by a good amount then only go up to 5 polynomial degrees instead of 10 as the model will overfit the data past a certain point.